# Compute the NLL metric before and after tracking

In [1]:
import numpy as np
import torch
import math
import os
import properscoring as ps
import warnings
warnings.filterwarnings('ignore')

In [2]:
before_path = "../det/check/check_loss_two_step_center_sr_ind/"
after_path = "./sort/cov/"
after_cfk_path = after_path + "cfk/"
after_nlla_path = after_path + "nlla/"
after_origin_path = after_path + "origin/"
after_cfk_nlla_path = after_path + "cfk_nlla/"
gt_path = "./TrackEval/data/gt/mot_challenge/"
scene_idxes_file_path = "../utils/test_scenes.txt"
var_cp_dict = {"upperbound": [125.44897552926587, 58.17835406418326, 224.21414415575381, 3907.8781994455294], "disco": [267.6059187072261, 65.37177933480551, 214.8165049926819, 4107.835669322297], "lowerbound": [339.5195216553872, 111.26186541465519, 298.26359427936194, 15076.497769642287]}

In [3]:
def convert_bbox_to_z(bbox):
    """
    Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
      [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
      the aspect ratio
    """
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    x = bbox[0] + w / 2.0
    y = bbox[1] + h / 2.0
    s = w * h  # scale is just area
    if float(h) < 0.00001:
        print([x, y, w, h])
        print(bbox)
    r = w / float(h)
    return np.array([x, y, s, r])


def convert_x_to_bbox(x, score=None):
    """
    Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
      [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
    """
    w = np.sqrt(x[2] * x[3])
    h = x[2] / w
    if score is None:
        return np.array(
            [x[0] - w / 2.0, x[1] - h / 2.0, x[0] + w / 2.0, x[1] + h / 2.0]
        ).reshape((1, 4))
    else:
        return np.array(
            [x[0] - w / 2.0, x[1] - h / 2.0, x[0] + w / 2.0, x[1] + h / 2.0, score]
        ).reshape((1, 5))

def split_data_by_frame(data):
    res = []
    start = 0
    if data.shape[0] == 0:
        return res
    if len(data.shape) == 1:
        res.append(data[np.newaxis, :])
        return res
    for i in range(1,data.shape[0]):
        if data[i][0] != data[start][0]:
            res.append(data[start:i,:])
            start = i
    res.append(data[start:,:])
    return res

def iou_batch(bb_test, bb_gt):
    """
    From SORT: Computes IOU between two bboxes in the form [x1,y1,x2,y2]
    """
    bb_gt = np.expand_dims(bb_gt, 0)
    bb_test = np.expand_dims(bb_test, 1)

    xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0])
    yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
    xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
    yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
    w = np.maximum(0.0, xx2 - xx1)
    h = np.maximum(0.0, yy2 - yy1)
    wh = w * h
    o = wh / (
        (bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])
        + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1])
        - wh
    )
    return o

def linear_assignment(cost_matrix):
    try:
        import lap

        _, x, y = lap.lapjv(cost_matrix, extend_cost=True)
        return np.array([[y[i], i] for i in x if i >= 0])  #
    except ImportError:
        from scipy.optimize import linear_sum_assignment

        x, y = linear_sum_assignment(cost_matrix)
        return np.array(list(zip(x, y)))

def pre_process_bbox(datas):
    """
    make the bbox be [x1,y1,x2,y2] where [x1,y1] is the left bottom and [x2,y2] is the right top
    """
    for i in range(datas.shape[0]):
        small = [min(datas[i][0], datas[i][2]), min(datas[i][1], datas[i][3])]
        large = [max(datas[i][0], datas[i][2]), max(datas[i][1], datas[i][3])]
        datas[i][0] = small[0]
        datas[i][1] = small[1]
        datas[i][2] = large[0]
        datas[i][3] = large[1]
    return datas

def remove_error_bbox(dets):
    res = []
    for i in range(dets.shape[0]):
        if dets[i, 0] == dets[i, 2] or dets[i, 1] == dets[i, 3]:
            continue
        res.append(list(dets[i]))
    return np.array(res)

def convert_bboxs_to_zs(datas):
    for i in range(datas.shape[0]):
        datas[i, :4] = convert_bbox_to_z(datas[i,:4])
    return datas
    
def compute_nll(dets, gts, mode, is_before_tracking, iou_threshold, is_use_cp_cov):
    dets = pre_process_bbox(dets[:,2:])
    gts = pre_process_bbox(gts[:,2:])
    dets = remove_error_bbox(dets)
    iou_matrix = iou_batch(dets, gts)
    if min(iou_matrix.shape) > 0:
        a = (iou_matrix > iou_threshold).astype(np.int32)
        if a.sum(1).max() == 1 and a.sum(0).max() == 1:
            matched_indices = np.stack(np.where(a), axis=1)
        else:
            matched_indices = linear_assignment(-iou_matrix)
    else:
        matched_indices = np.empty(shape=(0, 2))
    if len(matched_indices) == 0:
        print("Error length of matched_indices if zero!")
    if iou_threshold >= 0.5:
        matches = []
        for m in matched_indices:
            if iou_matrix[m[0], m[1]] >= iou_threshold:
                matches.append([m[0], m[1]])
        matched_indices = np.array(matches)
    if len(np.array(matched_indices).shape) < 2:
        return [], matched_indices
    matched_dets = dets[matched_indices[:,0]]
    matched_gts = gts[matched_indices[:,1]]
    pred = torch.from_numpy(convert_bboxs_to_zs(matched_dets[:,:4]))
    target = torch.from_numpy(convert_bboxs_to_zs(matched_gts[:,:4]))
    if is_before_tracking:
        cov = np.exp(matched_dets[:,8:])
    else:
        cov = matched_dets[:,8:]
    if is_use_cp_cov:
        cp_cov = np.array(var_cp_dict[mode])
        cov = cov * cp_cov
    cov = torch.from_numpy(cov)
    std = torch.sqrt(cov)
    predicted_normal_dists = torch.distributions.normal.Normal(pred, std)
    negative_log_prob = - predicted_normal_dists.log_prob(target)
    negative_log_prob = torch.clamp(negative_log_prob, min = 0)
    negative_log_prob = torch.sum(negative_log_prob,axis=1)/4
    return negative_log_prob.tolist(), matched_indices

def compute_nll_for_mode(mode, det_path, is_before_tracking, iou_threshold, is_use_cp_cov):
    upper_path = det_path + mode + "/no_rsu/"
    scene_idxes_file = open(scene_idxes_file_path, "r")
    scene_idxes = [int(line.strip()) for line in scene_idxes_file]
    #print(scene_idxes)
    nll_res = []
    for agent in range(1,6):
        det_files_path = upper_path + "tracking" + str(agent)
        gt_files_path = gt_path + "V2X-test" + str(agent)
        for scene in scene_idxes:
            if is_before_tracking:
                det_scene_file = os.path.join(det_files_path, "det_" + str(scene) + ".txt")
            else:
                det_scene_file = os.path.join(det_files_path, str(scene) + ".txt")
            gt_scene_file = os.path.join(gt_files_path, str(scene), "gt/gt.txt")
            det_datas = np.loadtxt(det_scene_file, delimiter=",")
            gt_datas = np.loadtxt(gt_scene_file, delimiter=",")
            if len(gt_datas) == 0 or len(det_datas) == 0:
                continue
            det_datas = split_data_by_frame(det_datas)
            gt_datas = split_data_by_frame(gt_datas)
            det_idx = 0
            for gt_idx in range(len(gt_datas)):
                if det_datas[det_idx][0][0] != gt_datas[gt_idx][0][0]:
                    #print(f"Error idx not match {scene} {agent} {det_idx}, {gt_idx} {det_datas[det_idx][0][0]} {gt_datas[gt_idx][0][0]}")
                    continue
                if gt_idx >= len(gt_datas):
                    print(f"gt_idx out of range, {gt_idx} {det_idx}")
                    break
                nll, matched_indices = compute_nll(det_datas[det_idx], gt_datas[gt_idx], mode, is_before_tracking, iou_threshold, is_use_cp_cov)
                # if nll == []:
                #     print(det_scene_file)
                #     print(det_datas[det_idx])
                #     return 0
                nll_res.extend(nll)
                det_idx += 1
    nll_mean = sum(nll_res) / len(nll_res)
    return nll_mean

def show_one_result(path, is_before_tracking=True, iou_threshold = 0.5, is_use_cp_cov=True):
    mode_list = ["disco", "upperbound","lowerbound"]
    for mode in mode_list:
        nll = compute_nll_for_mode(mode, path, is_before_tracking, iou_threshold, is_use_cp_cov)
        print(f"NLL for {mode} is {nll}")

## before tracking

In [4]:
mode = "upperbound"
nll = compute_nll_for_mode(mode, before_path, True, 0.3, True)
print(f"NLL for {mode} is {nll}")

NLL for upperbound is 1241038.315260995


In [5]:
show_one_result(before_path, True, iou_threshold=0.5, is_use_cp_cov=True)

NLL for disco is 18557.43558627593
NLL for upperbound is 14832.981308727705
NLL for lowerbound is 18433.28426202485


In [6]:
show_one_result(before_path, True, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 3987666.448754218
NLL for upperbound is 3332498.2558025816
NLL for lowerbound is 5496088.970781054


## after tracking

In [7]:
show_one_result(after_origin_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 2225.282155461875
NLL for upperbound is 2465.415435354346
NLL for lowerbound is 5334.005386521985


In [8]:
show_one_result(after_cfk_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 7216.887911809329
NLL for upperbound is 6558.770916062928
NLL for lowerbound is 13721.272217331854


In [9]:
show_one_result(after_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 2801.5782821221023
NLL for upperbound is 2807.5869690956056
NLL for lowerbound is 5582.613754514941


In [10]:
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 12641.110879345504
NLL for upperbound is 6948.51935019349
NLL for lowerbound is 13771.434507953049


# CRPS

In [11]:
def compute_crps(dets, gts, mode, is_before_tracking, iou_threshold, is_use_cp_cov):
    dets = pre_process_bbox(dets[:,2:])
    gts = pre_process_bbox(gts[:,2:])
    dets = remove_error_bbox(dets)
    iou_matrix = iou_batch(dets, gts)
    if min(iou_matrix.shape) > 0:
        a = (iou_matrix > iou_threshold).astype(np.int32)
        if a.sum(1).max() == 1 and a.sum(0).max() == 1:
            matched_indices = np.stack(np.where(a), axis=1)
        else:
            matched_indices = linear_assignment(-iou_matrix)
    else:
        matched_indices = np.empty(shape=(0, 2))
    if len(matched_indices) == 0:
        print("Error length of matched_indices if zero!")
    if iou_threshold >= 0.5:
        matches = []
        for m in matched_indices:
            if iou_matrix[m[0], m[1]] >= iou_threshold:
                matches.append([m[0], m[1]])
        matched_indices = np.array(matches)
    if len(np.array(matched_indices).shape) < 2:
        return [], matched_indices
    matched_dets = dets[matched_indices[:,0]]
    matched_gts = gts[matched_indices[:,1]]
    pred = convert_bboxs_to_zs(matched_dets[:,:4])
    target = convert_bboxs_to_zs(matched_gts[:,:4])
    if is_before_tracking:
        cov = np.exp(matched_dets[:,8:])
    else:
        cov = matched_dets[:,8:]
    if is_use_cp_cov:
        cp_cov = np.array(var_cp_dict[mode])
        cov = cov * cp_cov
    std = np.sqrt(cov)
    crps = ps.crps_gaussian(target, 
                               pred,
                               std)
    return np.mean(crps, axis=1).tolist(), matched_indices

def compute_crps_for_mode(mode, det_path, is_before_tracking, iou_threshold, is_use_cp_cov):
    upper_path = det_path + mode + "/no_rsu/"
    scene_idxes_file = open(scene_idxes_file_path, "r")
    scene_idxes = [int(line.strip()) for line in scene_idxes_file]
    #print(scene_idxes)
    nll_res = []
    for agent in range(1,6):
        det_files_path = upper_path + "tracking" + str(agent)
        gt_files_path = gt_path + "V2X-test" + str(agent)
        for scene in scene_idxes:
            if is_before_tracking:
                det_scene_file = os.path.join(det_files_path, "det_" + str(scene) + ".txt")
            else:
                det_scene_file = os.path.join(det_files_path, str(scene) + ".txt")
            gt_scene_file = os.path.join(gt_files_path, str(scene), "gt/gt.txt")
            det_datas = np.loadtxt(det_scene_file, delimiter=",")
            gt_datas = np.loadtxt(gt_scene_file, delimiter=",")
            if len(gt_datas) == 0 or len(det_datas) == 0:
                continue
            det_datas = split_data_by_frame(det_datas)
            gt_datas = split_data_by_frame(gt_datas)
            det_idx = 0
            for gt_idx in range(len(gt_datas)):
                if det_datas[det_idx][0][0] != gt_datas[gt_idx][0][0]:
                    #print(f"Error idx not match {scene} {agent} {det_idx}, {gt_idx} {det_datas[det_idx][0][0]} {gt_datas[gt_idx][0][0]}")
                    continue
                if gt_idx >= len(gt_datas):
                    print(f"gt_idx out of range, {gt_idx} {det_idx}")
                    break
                nll, matched_indices = compute_crps(det_datas[det_idx], gt_datas[gt_idx], mode, is_before_tracking, iou_threshold, is_use_cp_cov)
                # if nll == []:
                #     print(det_scene_file)
                #     print(det_datas[det_idx])
                #     return 0
                nll_res.extend(nll)
                det_idx += 1
    nll_mean = sum(nll_res) / len(nll_res)
    return nll_mean

def show_one_result_crps(path, is_before_tracking=True, iou_threshold = 0.5, is_use_cp_cov=True):
    mode_list = ["disco", "upperbound","lowerbound"]
    for mode in mode_list:
        nll = compute_crps_for_mode(mode, path, is_before_tracking, iou_threshold, is_use_cp_cov)
        print(f"NLL for {mode} is {nll}")

In [12]:
show_one_result_crps(before_path, True, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 50.89967227371934
NLL for upperbound is 50.357645926022656
NLL for lowerbound is 63.84297971033479


In [13]:
show_one_result_crps(before_path, True, iou_threshold=0.5, is_use_cp_cov=True)

NLL for disco is 50.686478888166384
NLL for upperbound is 50.13110200628221
NLL for lowerbound is 63.60137017239857


In [14]:
show_one_result_crps(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 37.71570296937291
NLL for upperbound is 36.632723263428126
NLL for lowerbound is 44.23447311139133
